In [8]:
from tqdm import tqdm
import simplejson as json
import requests
from pyquery import PyQuery as pq
from PIL import Image, ImageOps, ImageEnhance    # Pillow

import os
import urllib.parse
from glob import glob

In [15]:
autocontrastcutoff = 5   # Cut off how many percent of histogram?
imagesharpness = 1.2   # How much to sharpen?  Was 1.5

In [16]:
urlswanted = [
    "https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558#page/1/mode/2up"
]

In [17]:
urlpre = "https://essss.library.vanderbilt.edu/iiif/2/"
urlpost = "/full/pct:100/0/default.jpg"

In [18]:
headers = {
    "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "has_js=1; _ga=GA1.2.1752803547.1599356991; _gid=GA1.2.880756344.1599356991; _gat=1",
    "Host": "essss.library.vanderbilt.edu",
    "Referer": "https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558",
    "Sec-Fetch-Dest": "image",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "same-origin"
}

In [19]:
for startpage in urlswanted:
    r = requests.get(startpage)
    html = r.content
    bookname = pq(html)("li.active")[0].text
    bookdir = "books/" + bookname + "/"
    rawdir = bookdir + "raw/"
    processeddir = bookdir + "processed/"
    for mydir in [rawdir, processeddir]:
        os.makedirs(mydir, exist_ok=True)
    print(f"Processing {bookname}")
    rawjson = r.text.split(',"pageCount"')[0].split(',"pages":')[1]
    rawlist = json.loads(rawjson)
    for mypage in tqdm(rawlist[:]):
        targetpage = mypage['page'].zfill(3)
        targetfilename = rawdir + targetpage + ".jpg"
        targeturl = urlpre + urllib.parse.unquote(mypage['identifier']) + urlpost
        if not os.path.exists(targetfilename):
            r = requests.get(targeturl, headers=headers)
            if r.status_code == 200:
                with open(targetfilename, "wb") as outfile:
                    outfile.write(r.content)

100%|█████████████████████████████████████████████████████████████████████████████| 330/330 [00:00<00:00, 12669.18it/s]

Processing Livro de batismos de livres, 1774-1787


In [22]:
for bookdir in list(glob("books/*")):
    print(f"Processing {bookdir}")
    rawdir = bookdir + "/raw/"
    processeddir = bookdir + "/processed/"
    rawfiles = glob(rawdir + "*.*")
    for rawfile in tqdm(rawfiles):
        rawfile = rawfile.replace("\\", "/")
        targetfile = rawfile.replace("/raw/", "/processed/")
        if not os.path.exists(targetfile):
            im = Image.open(rawfile)
            if im.width > im.height:
                im = im.rotate(90, expand=True)
            im = ImageOps.autocontrast(im, cutoff=autocontrastcutoff)
            enhancer = ImageEnhance.Sharpness(im)
            im = enhancer.enhance(imagesharpness)
            im.save(targetfile)


  0%|                                                                                          | 0/330 [00:00<?, ?it/s]

Processing books\Livro de batismos de livres, 1774-1787


100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [02:29<00:00,  2.21it/s]
